Initialization Routines

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
#Hyper parameters

In [ ]:
image_shape = np.array([256,448])
batch_size = 32
num_batches = 2
disparity_levels = 16
EPOCHS = 4

Reset graph and set up regularizer


In [ ]:
tf.reset_default_graph() 
regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)
(imh,imw) = image_shape
#Define Inputs for the network
lx = tf.placeholder(tf.float32, (batch_size, imh, imw, 3), name= "left_image")
ly = tf.placeholder(tf.float32, (batch_size, imh, imw, 3),name= "right_image")
y = tf.placeholder(tf.float32, (batch_size,imh, imw),name= "disparity_image")


Generalized function for Relu based convolution followed by unary Operation

In [ ]:
def conv_bn_relu_layer_in(input_layer,inshape,outshape, filter_shape, stride, id):
    conv2d_W = tf.get_variable('conv'+str(id), regularizer = regularizer, initializer = tf.truncated_normal(shape=(3, 3, inshape, outshape), mean = mu, stddev = sigma))
    conv2d_b = tf.Variable(tf.zeros(outshape),name = 'bias'+str(id))
    s   = tf.nn.conv2d(input_layer, conv2d_W, strides=[1,stride, stride,1], padding='SAME') + conv2d_b
    s_a = tf.nn.relu(s)
    return s_a
#generalized function for getting unary features
def unary_res(indata, inchannels, outchannels, stride, id):
    conv2d_W = tf.get_variable('conv'+str(id), regularizer = regularizer, 
                               initializer = tf.truncated_normal(shape=(3, 3, inchannels, outchannels),
                                                                 mean = mu, stddev = sigma));
    conv2d_b = tf.Variable(tf.zeros(outchannels),name = 'bias'+str(id))
    s   = tf.nn.conv2d(indata, conv2d_W, strides=[1,stride, stride,1], padding='SAME') + conv2d_b
    s_a = tf.nn.relu(s)
    return s_a

Generate Unary Operations

In [ ]:
mu = 0
sigma = 0.1
def get_unary_features(input_tensor_batch):
    conv0 = conv_bn_relu_layer_in(input_tensor_batch,3,disparity_levels, [3, 3, 3, disparity_levels], 1, 100)
    n = 2;
    res_inp = conv0
    for i in range(n):
        conv1 = unary_res(res_inp, disparity_levels, disparity_levels, 1, i)
        tf.Print(conv1, [tf.shape(conv1)], message="This is conv1: ",summarize = 5)
        conv3 = unary_res(conv1, disparity_levels, disparity_levels, 1, 10+i) #+ res_inp
        res_inp = conv3
    #final stage of the Unary network        
    convu_W = tf.get_variable("unary_conv", initializer = tf.truncated_normal(shape=[3, 3, disparity_levels,disparity_levels], mean = mu, stddev = sigma), regularizer=regularizer)
    convu_b = tf.Variable(tf.zeros(disparity_levels))
    conv1   = tf.nn.conv2d(res_inp, convu_W, strides=[1, 1, 1, 1 ], padding='SAME') + convu_b

    return conv1
    
    

In [ ]:

def get_cost_volume(lx,ly):
    with tf.variable_scope('unary') as unary_scope:
        ufx = get_unary_features(lx)
        unary_scope.reuse_variables()
        ufy = get_unary_features(ly)
    u_shape = tf.shape(ufx)
    cvu = tf.concat([ufx,ufy], axis = 3)
    cvu_shape = tf.shape(cvu)
    return cvu
#disparity Estimate routine

def disparity_estimates_low(lx,ly):
    cvd = get_cost_volume(lx,ly)
    #get cost volume
    s = cvd
    #generate another volume of disparity levels*2
    for id in range(1):
        conv2d_W = tf.get_variable('oebyone'+str(id), initializer = tf.truncated_normal(shape=(1, 1, disparity_levels*2, disparity_levels*2), mean = mu, stddev = sigma),
                           regularizer=regularizer);
        conv2d_b = tf.Variable(tf.zeros(disparity_levels*2),name = 'onebyone'+str(id))
        s   = tf.nn.conv2d(s, conv2d_W, strides=[1,1,1,1], padding='SAME') + conv2d_b
        s = tf.nn.relu(s)
    
    #This is an FC layer implemented as 1x1 filter
    conv2d_W = tf.get_variable(name = 'onebyone'+str(id),  regularizer=regularizer , initializer = tf.truncated_normal(shape=(1, 1, disparity_levels*2, disparity_levels), mean = mu, stddev = sigma));
    conv2d_b = tf.Variable(tf.zeros(disparity_levels),name = 'onebyone'+str(id))
    
    s   = tf.nn.conv2d(s, conv2d_W, strides=[1,1, 1,1], padding='SAME') + conv2d_b 
    
    #Soft Argmax -
    neg_pred_d = -s;
    sfmin = tf.nn.softmax(neg_pred_d,axis = 3)
    sfshape = tf.shape(sfmin)
    for i in range(disparity_levels):
        sfmin1 = tf.concat([tf.scalar_mul(i,sfmin[:,:,:,i]) for i in range(disparity_levels)],axis=1)
    sfmin_tem = tf.reshape(sfmin1,[sfshape[0],sfshape[3],sfshape[1],sfshape[2]])
    sfmin2 = tf.reduce_sum(sfmin_tem, axis=1)
    return (sfmin2)


In [ ]:

#Estimate disparity and generate loss function
disp_est = disparity_estimates_low(lx,ly)
diff = tf.subtract( y, disp_est)
L2 = tf.multiply(diff,diff)

#compute regularization
reg_variables = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
reg_term = tf.contrib.layers.apply_regularization(regularizer, reg_variables)

#net loss
Loss = tf.reduce_mean(L2) + reg_term

#optimizer function and learning rate
optimizer = tf.train.AdamOptimizer(learning_rate = 0.0001)
training_operation = optimizer.minimize(Loss)


In [ ]:
import re
import numpy as np
import sys
def readPFM(file):
    file = open(file, 'rb')
    color = None
    width = None
    height = None
    scale = None
    endian = None
    header = file.readline().rstrip()
    if header == 'PF':
        color = True
    elif header == 'Pf':
        color = False
    else:
        raise Exception('Not a PFM file.')
    dim_match = re.match(r'^(\d+)\s(\d+)\s$', file.readline())
    if dim_match:
        width, height = map(int, dim_match.groups())
    else:
        raise Exception('Malformed PFM header.')
    scale = float(file.readline().rstrip())
    if scale < 0: # little-endian
        endian = '<'
        scale = -scale
    else:
        endian = '>' # big-endian
    data = np.fromfile(file, endian + 'f')
    shape = (height, width, 3) if color else (height, width)
    data = np.reshape(data, shape)
    data = np.flipud(data)
    return data, scale

In [ ]:

def normalize_values(limage, rimage, disparity):
    limage = np.array(limage).astype(float)/255.0
    rimage = np.array(rimage).astype(float)/255.0
    d = np.array(disparity)
    #print ("disparity max", np.amax(d))
    #print("disparity shape",d.shape)
    disparity = np.array(d).astype(float);
    return(limage,rimage,disparity)
    
import random
max_disp = 200
def get_image_disparity_for_index(filelist,indexandflip):
    index = indexandflip/2
    flipx = 0
    flipy = (indexandflip&0x1) 
    index = reduced_indices[index]
    limage = filelist[index].replace("disparity","pngs")[0:-5]+".png"
    rimage = limage.replace("left","right")
    limage_a = Image.open(limage)
    limg = np.array(limage_a.crop((0,0,image_shape[1]*2,image_shape[0]*2)).resize((image_shape[1],image_shape[0])))
    rimage_a = Image.open(rimage)
    rimg = np.array(rimage_a.crop((0,0,image_shape[1]*2,image_shape[0]*2)).resize((image_shape[1],image_shape[0])))
    pfmdata = readPFM(filelist[index][0:-1])
    resizepfm = np.array(pfmdata[0])[0:image_shape[0]*2:2, 0:image_shape[1]*2:2]/4
    
    if (np.amax(resizepfm) < max_disparity/4):
        limg = limage_a.crop((400,256,image_shape[1]+400,image_shape[0]+256))
        rimg = rimage_a.crop((400,256,image_shape[1]+400,image_shape[0]+256))
        resizepfm = np.array(pfmdata[0])[100:image_shape[0]+100, 200:image_shape[1]+200]
    if (flipx == 1):
        limg_temp = np.fliplr(limg)
        rimg_temp = np.fliplr(rimg)
        limg = rimg_temp
        rimg = limg_temp
        resizepfm = np.fliplr(resizepfm)
    if (flipy == 1):
        limg = np.flipud(limg)
        rimg = np.flipud(rimg)
        resizepfm = np.flipud(resizepfm)
    return (limg,rimg,resizepfm)
    
def get_disparity_images(filelist,indices):
    limages = []
    rimages = []
    disparities = []
    for index in indices:
        #print(filelist[index])
        (limage,rimage, disparity) = get_image_disparity_for_index(filelist,index)
        disparitynp = np.array(disparity).astype(float)
        #print("disparity", disparitynp.shape)
        limage,rimage,disparity = normalize_values(np.array(limage)[0:image_shape[0],0:image_shape[1],0:3],
                                                   np.array(rimage)[0:image_shape[0],0:image_shape[1],0:3], 
                                                   disparitynp[0:image_shape[0],0:image_shape[1]])
        limages.append(limage)
        rimages.append(rimage)
        disparities.append(disparity)
    return(limages,rimages,disparities)



In [ ]:
#file system related functions. Restrict calls to only the ones which matter

import os
import random
max_disp_allowed = 16
os.chdir("/home/ec2-user/project/data")
print(os.getcwd())
file = open("filelist","r")
filelist = file.readlines() 
all_indices = range(len(filelist))
random.shuffle(all_indices)
reduced_indices = []
for index in range(len(filelist)):
    pfmdata = readPFM(filelist[index][0:-1])
    if (index % 100 == 0):
        print("index ", index)
    resizepfm = np.array(pfmdata[0])[0:image_shape[0]*2:2, 0:image_shape[1]*2:2]/4
    
    if (np.amax(resizepfm) < max_disparity/4):
        resizepfm = np.array(pfmdata[0])[100:image_shape[0]+100, 200:image_shape[1]+200]
    if np.amax(resizepfm) <= max_disp_allowed:
        reduced_indices.append(index)


Evaluation network

In [ ]:
def evaluate(valbatch_lx, valbatch_ly, valbatch_dis):
    validation_accuracy = sess.run(Loss, feed_dict={lx: valbatch_lx, ly: valbatch_ly, y:valbatch_dis})
    return validation_accuracy
sess = tf.Session()
sess.run(tf.global_variables_initializer())

Training routine for the project

In [ ]:
max_disp =0
indices = range(len(reduced_indices)*2)
random.shuffle(indices)
training_indices = indices[-127:-1]
indices = indices[0:-127]
text_file = open("Output_disparity.txt", "w")
for i in range(EPOCHS):
    print("EPOCH", i)
    random.shuffle(indices)
    validation_indices = indices[-65:-1]
    for batch_index in range(int(len(indices)/batch_size)-1):
        (batch_lx, batch_ly, batch_dis) = get_disparity_images(filelist,indices[batch_index*batch_size:(batch_index+1)*batch_size])
        batch_dis = np.array(batch_dis).astype(float)
        print(" disparity max", np.amax(batch_dis))
        _,loss = sess.run([training_operation,Loss], feed_dict={lx: batch_lx, ly: batch_ly, y:batch_dis})
        random.shuffle(validation_indices)
        (valbatch_lx, valbatch_ly, valbatch_dis) = get_disparity_images(filelist,validation_indices[0:batch_size])
        validation_accuracy = evaluate(valbatch_lx, valbatch_ly, valbatch_dis)
        print("Training Loss = {:.3f}".format(loss))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        text_file.write("Training Loss = {:.3f}".format(loss))
        text_file.write("Validation Loss = {:.3f}".format(validation_accuracy))
saver = tf.train.Saver()
saver.save(sess, './disparity_simple')
print("Model saved")



# Utility code for changing format
os.chdir("/Users/bsubhash/Stanford_courses/Computer Vision/Project")
print(os.getcwd())
file = open("./data/filelist","r")
filelist = file.readlines() 
for index in range(len(filelist)):
    #print(index)
    limage_wp = filelist[index].replace("disparity","frames_cleanpass_webp")[0:-1]+".webp"
    #print(limage_wp)
    rimage_wp = limage_wp.replace("left","right")
    #print(rimage_wp)
    limage = (np.array(webp_read("./data/" + limage_wp)))
    rimage = (np.array(webp_read("./data/" + rimage_wp)))
    limage_png = "./data/" + limage_wp.replace(".webp",".png")
    rimage_png = "./data/" + rimage_wp.replace(".webp",".png")
    lim = Image.fromarray(limage[:,:,0:3])
    rim = Image.fromarray(rimage[:,:,0:3])
    lim.save(limage_png)
    rim.save(rimage_png)
    #plt.imshow(Image.open(rimage_png))
    
    
    

In [ ]:
#Test Routine
new_saver = tf.train.import_meta_graph('disparity_simple' + '.meta', clear_devices=True)
(limage,rimage, disparity) = get_disparity_images(filelist,training_indices[0:batch_size])
new_saver.restore(sess, 'disparity_simple')
dis_inf = sess.run(disp_est,feed_dict={lx:limage,ly:rimage})


In [ ]:
#get some interesting Stats:
error = np.abs(disparity-dis_inf)
print("greater than 5% error",100*np.sum(error>5).astype(int)/np.sum(error<15).astype(float))
print("greater than 5% error",100*np.sum(error>2).astype(int)/np.sum(error<15).astype(float))
print("greater than 5% error",100*np.sum(error>1).astype(int)/np.sum(error<15).astype(float))